In [37]:
#โฟลเดอร์
import os
import pandas as pd
from tqdm import tqdm

def read_files_in_fraction(directory_path, fraction_start, fraction_end):
    # Get a list of file names in the directory
    file_names = os.listdir(directory_path)

    # Calculate the number of files to read based on the fraction
    num_files_to_read_start = int(len(file_names) * fraction_start)
    num_files_to_read_end = int(len(file_names) * fraction_end)

    # Initialize an empty list to store individual DataFrames
    dataframes = []

    # Use tqdm to create a progress bar for reading files
    for file_name in tqdm(file_names[num_files_to_read_start:num_files_to_read_end], desc="Reading files", unit="file"):
        if file_name.endswith('.csv'):  # Adjust the file extension based on your files (e.g., .csv, .xlsx, etc.)
            file_path = os.path.join(directory_path, file_name)
            df2 = pd.read_csv(file_path)  # Replace 'read_csv' with 'read_excel' for Excel files
            dataframes.append(df2)

    # Concatenate all the individual DataFrames into a single DataFrame
    df3 = pd.concat(dataframes, ignore_index=True)

    return df3

# Replace 'directory_path' with the path to the directory containing your files
directory_path = 'E:\\CUSC Datalake\\cham4'  #<<<<<<<<<<<<<

# Read 1/4 of the files
df = read_files_in_fraction(directory_path, 0.75, 1)  #<<<<<<<<<<<<<

file_save_name = r"E:\Sort Data\cham4_temp\cham4_temperature_4.csv"  #<<<<<<<<<<<<<

# Filter rows where "id" column contains "multi_sensor"
df = df[df['id'].str.contains('temperature')]

# Convert 'value' column to numeric (float) type if it's not already done
df['value'] = pd.to_numeric(df['value'], errors='coerce')

# Drop rows with value 0 in the "value" column
df = df[df['value'] != 0]

#extract id_floor
df['id_floor'] = df['id'].str.extract(r'(fl(?:m|\d{1,2}[a-zA-Z]?))')

#sort
df = df.sort_values(by=['id_floor','time']) 

# Remove the 'id' column
df = df.drop('id', axis=1)

# Convert 'time' column to pandas datetime object
df['time'] = pd.to_datetime(df['time'])

# Round the minute to the nearest minute
df['time'] = df['time'].dt.round('1min')

# Export the result to a CSV file
df.to_csv(file_save_name, index=False)

df = pd.read_csv(file_save_name)

# Group by 'time' and calculate the average temperature for each unique timestamp
df['value'] = df.groupby(['id_floor', 'time'])['value'].transform('mean')

# Format the 'value' column to two decimal places
df['value'] = df['value'].round(2)

# Drop rows with the same time
df = df.drop_duplicates(subset=['time', 'id_floor'], keep='first')

# rename
df.rename(columns={'value': 'temperature'}, inplace=True)

df.to_csv(file_save_name, index=False)



Reading files: 100%|██████████| 732/732 [00:54<00:00, 13.45file/s]


In [1]:
#หาไอดีไม่ซ้ำ
import os
import pandas as pd
from tqdm import tqdm

df = pd.read_csv(r"E:\CUSC Datalake\cham4\cham4-2022-09-01T0800.csv")

unique_id_column = df.drop_duplicates(subset='id')['id']

unique_id_column.to_csv('E:\\Sort Data\\cham4_id.csv', index=False)

: 

In [41]:
# รวม energy & temperature

import os
import pandas as pd
from tqdm import tqdm
from datetime import datetime

# def read_files_in_fraction(directory_path, fraction_start, fraction_end):
#     # Get a list of file names in the directory
#     file_names = os.listdir(directory_path)

#     # Calculate the number of files to read based on the fraction
#     num_files_to_read_start = int(len(file_names) * fraction_start)
#     num_files_to_read_end = int(len(file_names) * fraction_end)

#     # Initialize an empty list to store individual DataFrames
#     dataframes = []

#     # Use tqdm to create a progress bar for reading files
#     for file_name in tqdm(file_names[num_files_to_read_start:num_files_to_read_end], desc="Reading files", unit="file"):
#         if file_name.endswith('.csv'):  # Adjust the file extension based on your files (e.g., .csv, .xlsx, etc.)
#             file_path = os.path.join(directory_path, file_name)
#             df2 = pd.read_csv(file_path)  # Replace 'read_csv' with 'read_excel' for Excel files
#             dataframes.append(df2)

#     # Concatenate all the individual DataFrames into a single DataFrame
#     df3 = pd.concat(dataframes, ignore_index=True)

#     return df3

# # Replace 'directory_path' with the path to the directory containing your files
# directory_path = 'E:\\CUSC Datalake\\cham4'  #<<<<<<<<<<<<<

# # Read 1/4 of the files
# df1 = read_files_in_fraction(directory_path, 0.25, 0.5)  #<<<<<<<<<<<<<

#read files
df1 = pd.read_csv(r"E:\Sort Data\cham4_temp\cham4_temperature_4.csv")  #<<<<<<<<<<<<<<
df2 = pd.read_csv(r"E:\Sort Data\cham4_temp\cham4_temperature_4.csv")  #<<<<<<<<<<<<<<

# Convert 'time' columns to datetime objects for both DataFrames
df1['time'] = pd.to_datetime(df1['time'])
df2['time'] = pd.to_datetime(df2['time'])

# Sort both DataFrames by 'time'
df1 = df1.sort_values(by='time')
df2 = df2.sort_values(by='time')

#rename columns
df1.rename(columns={'value': 'energy'}, inplace=True)
df1.rename(columns={'id_group': 'id_floor'}, inplace=True)

# Merge the two DataFrames based on 'time' and 'id_floor' using merge_asof
result = pd.merge_asof(df1, df2, on='time', by='id_floor', direction='forward', tolerance=pd.Timedelta('5 minutes'))

#sort
result = result.sort_values(by=['id_floor','time'])

print(result)

result.to_csv(r"E:\Sort Data\cham4_energy&temp_4.csv", index=False)  #<<<<<<<<<<<<<<

       id_floor                      time  energy  temperature
0           fl1 2022-09-10 09:00:00+08:00   2.525        28.90
12          fl1 2022-09-10 09:01:00+08:00   2.592        28.90
15          fl1 2022-09-10 09:02:00+08:00   2.553        28.90
22          fl1 2022-09-10 09:03:00+08:00   4.292        28.90
30          fl1 2022-09-10 09:04:00+08:00  12.456        28.90
...         ...                       ...     ...          ...
282284      fl7 2022-12-31 23:55:00+08:00  60.444        29.41
282292      fl7 2022-12-31 23:56:00+08:00  60.462        29.89
282302      fl7 2022-12-31 23:57:00+08:00  60.355        29.89
282308      fl7 2022-12-31 23:58:00+08:00  55.183        29.51
282316      fl7 2022-12-31 23:59:00+08:00  53.679        29.28

[282317 rows x 4 columns]


In [1]:
import os
import pandas as pd
from tqdm import tqdm
from datetime import datetime

a=r"E:\Sort Data\cham4_mix\cham4_4variable_4.csv"

df = pd.read_csv(a)

#df = df.sort_values(by=['id_floor','time'])

df.rename(columns={'value': 'energy'}, inplace=True)
#df.rename(columns={'id_group': 'id_floor'}, inplace=True)

#Count the number of empty energy and temperature values
#num_empty_energy = df['energy'].isnull().sum()
#num_empty_temperature = df['temperature'].isnull().sum()

#print(f"Number of empty energy values: {num_empty_energy}")
#print(f"Number of empty temperature values: {num_empty_temperature}")

df.to_csv(a, index=False)

In [16]:
import os
import pandas as pd
from tqdm import tqdm
from datetime import datetime

df1 = pd.read_csv(r"E:\Sort Data\cham4_mix\cham4_4variable_4.csv")
df2 = pd.read_csv(r"E:\Sort Data\cham4_intensity\cham4_intensity_4.csv")

# Convert the 'time' column to datetime type
df1['time'] = pd.to_datetime(df1['time'])
df2['time'] = pd.to_datetime(df2['time'])

# Print unique values in 'time' and 'id_floor' columns in both DataFrames
print("Unique 'time' values in df1:")
print(df1['time'].unique())
print("\nUnique 'time' values in df2:")
print(df2['time'].unique())

print("\nUnique 'id_floor' values in df1:")
print(df1['id_floor'].unique())
print("\nUnique 'id_floor' values in df2:")
print(df2['id_floor'].unique())

# Perform an SQL-style merge (join) on 'time' and 'id_floor' columns
merged_df = pd.merge(df1, df2, on=['time', 'id_floor'], how='outer')

# Filter rows where the 'time' and 'id_floor' are not the same in both dataframes
different_times_and_floor_df = merged_df[~(merged_df['time'] == merged_df['time'])]

# Drop the additional columns used for the merge (optional)
different_times_and_floor_df.drop(['energy'], axis=1, inplace=True)

# Display the result
print(different_times_and_floor_df)

Unique 'time' values in df1:
<DatetimeArray>
['2022-09-10 09:00:00+08:00', '2022-09-10 09:01:00+08:00',
 '2022-09-10 09:02:00+08:00', '2022-09-10 09:03:00+08:00',
 '2022-09-10 09:04:00+08:00', '2022-09-10 09:05:00+08:00',
 '2022-09-10 09:06:00+08:00', '2022-09-10 09:07:00+08:00',
 '2022-09-10 09:08:00+08:00', '2022-09-10 09:09:00+08:00',
 ...
 '2022-12-18 22:46:00+08:00', '2022-12-19 03:01:00+08:00',
 '2022-12-19 08:16:00+08:00', '2022-12-19 10:01:00+08:00',
 '2022-12-19 11:16:00+08:00', '2022-12-19 13:41:00+08:00',
 '2022-12-19 13:46:00+08:00', '2022-12-20 11:16:00+08:00',
 '2022-12-22 12:16:00+08:00', '2022-12-29 15:13:00+08:00']
Length: 40345, dtype: datetime64[ns, UTC+08:00]

Unique 'time' values in df2:
<DatetimeArray>
['2022-09-10 09:03:00+08:00', '2022-09-10 09:08:00+08:00',
 '2022-09-10 09:13:00+08:00', '2022-09-10 09:17:00+08:00',
 '2022-09-10 09:18:00+08:00', '2022-09-10 09:23:00+08:00',
 '2022-09-10 09:28:00+08:00', '2022-09-10 09:33:00+08:00',
 '2022-09-10 09:38:00+08:00', 

C:\Users\ASUS\AppData\Local\Temp/ipykernel_13500/2701108392.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  different_times_and_floor_df.drop(['energy'], axis=1, inplace=True)
